In [1]:
import pykafka
import json
import tweepy
from tweepy import OAuthHandler
from tweepy import Stream
from datetime import datetime,timezone
from tweepy.streaming import StreamListener
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

#TWITTER API CONFIGURATIONS
consumer_key = 'mfVmUQjDKblS4AC5iSy05yvPX'
consumer_secret = 'dB7AZn8WzY362xOSVLYLCOsqYHkwPUz3taYHHVrBN0ovMNguyH'
access_token = '1307663649519988736-u7v0XWCNfRFmhWXZd87SuKAcc88cze'
access_secret = 'Jn3oU9j68wJTLsdWHewKvzeuFcmKG5P3DEPzgbSWit6y8'

#TWITTER API AUTH
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth)

SA = SentimentIntensityAnalyzer()

def clean_tweet_extract(data):
    
    rawtweet = json.loads(data)
    tweet={}
    tweet["date"] = datetime.strptime(rawtweet["created_at"],'%a %b %d %H:%M:%S %z %Y')\
        .replace(tzinfo=timezone.utc).astimezone(tz=None).strftime('%Y-%m-%d %H:%M:%S')
    tweet["user"] = rawtweet["user"]["screen_name"]
    if "extended_tweet" in rawtweet:
        tweet["text"] = rawtweet["extended_tweet"]["full_text"]
    else:
        tweet["text"] = rawtweet["text"]
        
    #tweet["country"] = rawtweet['place']['country']
    #tweet['country_code'] = rawtweet['place']['country_code']
    
    tweet['reply_count'] = rawtweet['reply_count']
    tweet['retweet_count'] = rawtweet['retweet_count']
    tweet['favorite_count'] = rawtweet['favorite_count']
    #Sentiment Analysis
    sentiment_dict = SA.polarity_scores(tweet['text'])
    tweet['sentiment_score'] = sentiment_dict['compound']
    
    return json.dumps(tweet)

#Twitter Stream Listener
class KafkaPushListener(StreamListener):          
    def __init__(self):
        #localhost:9092 = Default Zookeeper Producer Host and Port Adresses
        self.client = pykafka.KafkaClient("localhost:9092")

        #Get Producer that has topic name is Twitter
        self.producer = self.client.topics[bytes("kafkaspark", "ascii")].get_producer()
  
    def on_data(self, data):
        #Producer produces data for consumer
        #Data comes from Twitter
        processed_data = clean_tweet_extract(data)
        self.producer.produce(bytes(processed_data, "ascii"))
        print(processed_data)
        return True
                                                                                                                                           
    def on_error(self, status):
        print(status)
        return True

#Twitter Stream Config
twitter_stream = Stream(auth, KafkaPushListener())

#Produce Data that has Game of Thrones hashtag (Tweets)
twitter_stream.filter(track=['#Trump'])

{"date": "2020-11-25 15:49:53", "user": "AnitaBeavers", "text": "Nuke China making sense for     #Trump", "reply_count": 0, "retweet_count": 0, "favorite_count": 0, "sentiment_score": 0.0}
{"date": "2020-11-25 15:49:53", "user": "MythicalStig", "text": "@YayImNotaBot @dccc1055 @MollyJongFast Key word in your statement is IDIOTS. No, the idiots don't recognize that more Americans voted against #Trump TWICE  (2016 AND 2020) than voted for him.", "reply_count": 0, "retweet_count": 0, "favorite_count": 0, "sentiment_score": 0.0}
{"date": "2020-11-25 15:49:55", "user": "funkygy123", "text": "RT @HugoSavage4: We are watching a big-mouthed criminal gasbag disappear before our very eyes. #Trump is getting smaller every day. \n#Trump\u2026", "reply_count": 0, "retweet_count": 0, "favorite_count": 0, "sentiment_score": -0.6486}
{"date": "2020-11-25 15:49:55", "user": "degiltaxi", "text": "RT @degiltaxi: @damn_commie @4freekorea #Trump #BIDEN\n#USA\n#\ubbf8\uad6d\ub300\uc120\n#\ubd80\uc815\uc120\

{"date": "2020-11-25 15:51:00", "user": "LDGay1", "text": "RT @SoHereForEver: &gt; 75% des dons pour \u00ab lutter contre la fraude \u00e9lectorale\u00bb partent en r\u00e9alit\u00e9 dans un PAC au b\u00e9n\u00e9fice de #Trump... \nApr\u00e8s\u2026", "reply_count": 0, "retweet_count": 0, "favorite_count": 0, "sentiment_score": 0.0}
{"date": "2020-11-25 15:51:01", "user": "LaughOutNOW", "text": "Veterans, Recovering Addicts, Anxiety ridden people or if you need a great laugh check this indie film out: https://t.co/LP5wgKQyx1 #trump #washingtondc #broadman #tacoma #sanjose #sanmateo #seattle #sacramento #larams #rays #bears", "reply_count": 0, "retweet_count": 0, "favorite_count": 0, "sentiment_score": 0.7906}
{"date": "2020-11-25 15:51:02", "user": "Annonymost", "text": "@Paula_White The hottest place in \"Hell\" is reserved for those who use religion, exploiting suffering people, for personal gain. By giving #trump some religious validity, you took advantage of the lunatic for your per

{"date": "2020-11-25 15:51:47", "user": "bitcoinconnect", "text": "The Best Things to Buy on Black Friday That Would Make Great Holiday Gifts SEE MORE HERE ==&gt; https://t.co/2CVGAJFLHG  #blackfriday #covid19 #covid #coronavirus #trump https://t.co/Dhe59lrghZ", "reply_count": 0, "retweet_count": 0, "favorite_count": 0, "sentiment_score": 0.9001}
{"date": "2020-11-25 15:51:48", "user": "BLUIZK81", "text": "RT @pinkk9lover: Did you know...8% of the #Biden supporters that DIDNT know about #OperationWarpSpeed said if they had known- they would ha\u2026", "reply_count": 0, "retweet_count": 0, "favorite_count": 0, "sentiment_score": 0.4404}
{"date": "2020-11-25 15:51:48", "user": "marccannes2", "text": "RT @GWGoldnadel: Sans parler des sondages bidonn\u00e9s. https://t.co/N5TMLRkSgN", "reply_count": 0, "retweet_count": 0, "favorite_count": 0, "sentiment_score": 0.0}
{"date": "2020-11-25 15:51:50", "user": "juggernaut_red", "text": "RT @XianDa592351909: @KellzBellzzah @TrumpWarRoom @realDona

{"date": "2020-11-25 15:52:40", "user": "Hiber94314036", "text": "RT @htommy998: 11/24\ud83d\udca5\ud83d\udca5\u8db3\u591f\u591a\u7684\u6cd5\u5f8b\u5de5\u5177\uff0c\u8ba9\u5ddd\u666e\u603b\u7edf\u76ee\u524d\u5177\u6709\u8d85\u8d8a\u6cd5\u5f8b\u8d85\u5f3a\u5927\u7279\u6743\u300212/2017\u884c\u653f\u547d\u4ee4\u5ba3\u5e03\u56fd\u5bb6\u7d27\u6025\u72b6\u6001\uff0c\u5bf9\u8feb\u5bb3\u4eba\u6743\u548c\u8150\u8d25\u8005\u53ef\u4ee5\u6ca1\u6536\u8d22\u4ea7\uff1b7/2012 \u5965\u5df4\u7684\u56fd\u5bb6\u5b89\u5168\u548c\u5e94\u6025\u51c6\u5907\u901a\u4fe1\u804c\u80fd\u7684\u5206\u914d\u547d\u4ee4\uff0c\u7ed9\u5ddd\u666e\u5b8c\u5168\u63a7\u5236\u6240\u6709\u7f8e\u56fd\u5a92\u4f53MSM\u7684\u6743\u529b\uff1b3/20\u2026", "reply_count": 0, "retweet_count": 0, "favorite_count": 0, "sentiment_score": -0.3612}
{"date": "2020-11-25 15:52:40", "user": "shazia73272395", "text": "RT @iamkatemiller: Maybe #Trump enjoys the turkey pardoning so much because he knows that HE\u2019s a turkey that needs pardoned. 

{"date": "2020-11-25 15:53:53", "user": "EZDay4U", "text": "@cspan INO4800 is the ONLY vaccine that requires no deep freezing and has no adverse side effects.  Why halt it?  Inovio has shown to defeat glioblastoma cancer too.  Lift the unjust halt now. #trump #health #COVID19 #inovio #dna #savelives #news #actnow #Trending https://t.co/84uema4gRW", "reply_count": 0, "retweet_count": 0, "favorite_count": 0, "sentiment_score": -0.8884}
{"date": "2020-11-25 15:53:59", "user": "SteveCo89062225", "text": "RT @zeta_mara: WOW! I KNEW Rachel #Maddow would be covering the Open Skies treaty. We all know Russia played games. But #Trump has done, ye\u2026", "reply_count": 0, "retweet_count": 0, "favorite_count": 0, "sentiment_score": 0.5801}
{"date": "2020-11-25 15:54:00", "user": "TheBalladOfira", "text": "RT @adrenaline1001: La strat\u00e9gie gagnante de #Trump\n\nLes 1ers r\u00e9sultats ont convaincu le general attorney William P. Barr qu\u2019un dossier d\u2019i\u2026", "reply_count": 0, "retw

...

{"date": "2020-11-25 18:56:16", "user": "KoldoSalazar", "text": "RT @geoestratego: Del \u201cAmerica first\u201d de #Trump al \u201cAmerica again\u201d para \u201cliderar el mundo\u201d de #Biden \u201cel pacificador\u201d.\nPrimer golpe contra\u2026", "reply_count": 0, "retweet_count": 0, "favorite_count": 0, "sentiment_score": 0.0}
{"date": "2020-11-25 18:56:17", "user": "louise_mireya", "text": "DOJ set to execute 5 federal prisoners before Inauguration Day - CNN Politics   Another #Trump #Murder statement to show his power to do as he pleases, at any cost.  Heinous, last ditch effort to show his strength to do as he pleases.  #Winning ?  https://t.co/vaTpnUcYIR", "reply_count": 0, "retweet_count": 0, "favorite_count": 0, "sentiment_score": 0.4588}
{"date": "2020-11-25 18:56:17", "user": "FrontalArmy", "text": "https://t.co/Or3zHuywRX", "reply_count": 0, "retweet_count": 0, "favorite_count": 0, "sentiment_score": 0.0}
{"date": "2020-11-25 18:56:17", "user": "Christo45046444", "text"

{"date": "2020-11-25 18:56:45", "user": "ljs1479", "text": "RT @TribulationThe: ANTIFA gearing up for the Biden loss! \n\nANTIFA ISSUES PUBLIC THREAT AGAINST TRUMP VOTERS\n\n#Trump #MAGA #TWGRP #TheMight\u2026", "reply_count": 0, "retweet_count": 0, "favorite_count": 0, "sentiment_score": -0.7734}
{"date": "2020-11-25 18:56:45", "user": "t1996er", "text": "RT @Tombx7M: Totally unwatchable\nAny thoughts?\n\n #DumpFoxNews #Newsmax #FridayFeeling \n#MAGA #Trump #KAG #StopTheSteaI \n #VoterFraud #WalkA\u2026", "reply_count": 0, "retweet_count": 0, "favorite_count": 0, "sentiment_score": 0.0}
{"date": "2020-11-25 18:56:45", "user": "Yellow34950210", "text": "#Israeli military prepares for possibility #Trump will strike #Iran https://t.co/uc5cfqQskj", "reply_count": 0, "retweet_count": 0, "favorite_count": 0, "sentiment_score": -0.128}
{"date": "2020-11-25 18:56:46", "user": "harleypyrate02", "text": "SCREW DOUBTERS!\n\nGLAD TO SEE WHOS DOWN TO FIGHT AGAINST TYRANNY!", "reply_count": 0, "ret

{"date": "2020-11-25 18:57:18", "user": "jblosser13", "text": "Trump is like a two year old throwing a tantrum because he didn\u2019t get his way.  The Dems would have stopped this if a Dem president was acting like such a moron.  #CountryOverParty.  Maybe we should start saying #CountryOverTrump.", "reply_count": 0, "retweet_count": 0, "favorite_count": 0, "sentiment_score": -0.4404}
{"date": "2020-11-25 18:57:21", "user": "Susan55107566", "text": "RT @htommy998: 11/24\ud83d\udca5\ud83d\udca5\u8db3\u591f\u591a\u7684\u6cd5\u5f8b\u5de5\u5177\uff0c\u8ba9\u5ddd\u666e\u603b\u7edf\u76ee\u524d\u5177\u6709\u8d85\u8d8a\u6cd5\u5f8b\u8d85\u5f3a\u5927\u7279\u6743\u300212/2017\u884c\u653f\u547d\u4ee4\u5ba3\u5e03\u56fd\u5bb6\u7d27\u6025\u72b6\u6001\uff0c\u5bf9\u8feb\u5bb3\u4eba\u6743\u548c\u8150\u8d25\u8005\u53ef\u4ee5\u6ca1\u6536\u8d22\u4ea7\uff1b7/2012 \u5965\u5df4\u7684\u56fd\u5bb6\u5b89\u5168\u548c\u5e94\u6025\u51c6\u5907\u901a\u4fe1\u804c\u80fd\u7684\u5206\u914d\u547d\u4ee4\uff0c\u7ed9\u5ddd\u

{"date": "2020-11-25 18:57:55", "user": "SLnTb", "text": "RT @aysen_tok: Vir\u00fcs\u00fc \u00e7\u0131karan #\u00c7in\nK\u0131sa s\u00fcrede vir\u00fcsten kurtulan #\u00c7in\nA\u015f\u0131y\u0131 t\u00fcm \u00fclkelerden \u00f6nce bulup, d\u00fcnyaya pazarlayan yine #\u00c7in\nD\u00fcnya p\u2026", "reply_count": 0, "retweet_count": 0, "favorite_count": 0, "sentiment_score": 0.0}
{"date": "2020-11-25 18:57:59", "user": "LuBonLez", "text": "In four short years, #Trump cut taxes for the rich, rolled back regulations that had protected our environment, alienated our allies, coddled dictators, oversaw the deaths of hundreds of thousands of Americans from a pandemic he failed to manage, and took children from parents.", "reply_count": 0, "retweet_count": 0, "favorite_count": 0, "sentiment_score": 0.2732}
{"date": "2020-11-25 18:57:59", "user": "Lunaticf69", "text": "RT @FrontalArmy: #Breaking: Israeli military prepares for possibility #US president #Trump will strike #Iran.\nhttps://t.co/aSO

KeyboardInterrupt: 

In [1]:
from platform import python_version

print(python_version())

3.6.7
